In [1]:
import Py_files.twcs_processor as processor
import pandas as pd
import os

In [2]:
user_input = "My Echo keeps playing the same song over and over, how do I fix recommendations?"

In [3]:
user_input = processor.TWCSProcessor._clean_single(user_input)
user_input_processed = processor.TWCSProcessor._convert_to_conversation(user_input)

In [4]:
# convert the user input to a pandas dataframe
user_input_df = pd.DataFrame([[user_input_processed]], columns=['cleaned_conversations'])

In [6]:
from Py_files.llm_extractor import LLMExtractor
pipe = LLMExtractor(dataframe = user_input_df)

# only products / issue-types / services
df1 = pipe.extract_entities()

# pack them into a single JSON field
df2 = pipe.process_entities_json()

# create RDF triples
df3 = pipe.extract_relationships()

# produce suggested resolutions
df4 = pipe.extract_resolution()

2025-05-11 13:03:48,596 [INFO] LLMExtractor: Loaded data – 1 rows
2025-05-11 13:03:48,597 [INFO] LLMExtractor: STEP 1 – Extracting issue-types, products, services
LLM steps: 100%|██████████| 1/1 [00:00<00:00,  1.41it/s]
2025-05-11 13:03:52,856 [INFO] LLMExtractor: STEP 2 – Packing entities into single JSON field
LLM steps: 100%|██████████| 1/1 [00:00<?, ?it/s]
2025-05-11 13:03:52,858 [INFO] LLMExtractor: STEP 3 – Extracting relationships (RDF triples)
LLM steps: 100%|██████████| 1/1 [00:02<00:00,  2.15s/it]
2025-05-11 13:03:55,015 [INFO] LLMExtractor: STEP 4 – Generating resolution completions
LLM steps: 100%|██████████| 1/1 [00:04<00:00,  4.00s/it]


In [7]:
df4

,cleaned_conversations,Issue Type,Product,Services,entities,relationship,resolution
0,"{'conversation': [{'role': 'Customer', 'messag...","{\n ""Company_name"": ""company_name_here"",\n ""...","{\n ""Company_name"": ""company_name_here"",\n ""...","{\n ""Company_name"": ""company_name_here"",\n ""...","{""products"": [""Echo""], ""services"": [], ""issue_...","[\n {""subject"": ""Echo"", ""predicate"": ""hasIs...","Based on the provided conversation, the custom..."


In [8]:
from Notebooks.VectorDBStructure.db_structure import DatabaseStructure

db = DatabaseStructure()

cleaned_conversations = user_input_processed
entities = df4['entities'].values[0]
resolution = df4['resolution'].values[0]
relationship = df4['relationship'].values[0]

fixed_relationships = db.fix_relationships(relationship,resolution)

C:\Users\paris\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-05-11 13:07:40,415 [INFO] datasets: PyTorch version 2.1.1+cu121 available.
2025-05-11 13:07:40,417 [INFO] datasets: TensorFlow version 2.12.1 available.
2025-05-11 13:07:41,022 [INFO] sentence_transformers.SentenceTransformer: Use pytorch device_name: cuda
2025-05-11 13:07:41,022 [INFO] sentence_transformers.SentenceTransformer: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [9]:
fixed_relationships

'[{"subject": "Echo", "predicate": "hasIssue", "object": "repeating song issue"},{"subject": "Echo", "predicate": "hasIssue", "object": "recommendation problem"}]'

In [11]:
db.text_to_embedding(entities,fixed_relationships)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([-8.84778704e-03, -3.15466188e-02, -4.00599800e-02, -5.85404271e-03,
       -7.18602631e-03,  8.27535391e-02,  1.26421258e-01, -3.50641422e-02,
       -1.07414030e-01, -8.96736830e-02,  2.26788186e-02, -1.73727572e-02,
       -7.97864888e-03, -3.31071727e-02, -4.21000319e-03,  6.90747127e-02,
       -7.24512246e-03, -1.04118086e-01,  1.42557425e-02, -8.18298981e-02,
       -1.50931086e-02,  6.67137131e-02,  1.53139438e-02,  6.18973598e-02,
       -1.02656178e-01,  3.54813337e-02, -6.37803823e-02,  5.82646625e-03,
       -4.12987992e-02, -7.27533177e-02,  3.16500068e-02,  1.36102766e-01,
        2.35700980e-02, -6.68299422e-02,  1.02966174e-01, -3.67101771e-03,
       -1.81632247e-02, -9.40286741e-03, -5.04894853e-02, -2.25453340e-02,
        2.98011359e-02, -2.96211224e-02, -3.45389172e-02, -9.70606506e-03,
       -7.78992772e-02, -1.15345091e-01, -9.25213248e-02, -4.02807668e-02,
        7.56505355e-02,  3.72028182e-04, -4.75274622e-02,  4.77132993e-03,
        3.97746563e-02,  